In [8]:
# Prosty SQL Agent dla analizy logów sieciowych
# Jupyter Notebook

# 1. Imports i konfiguracja
import os
import sqlite3
import pandas as pd
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain.agents.agent_types import AgentType

# Załaduj zmienne środowiskowe
load_dotenv()

# 2. Konfiguracja
class Config:
    DB_PATH = ".
    ./parser/logs.db"  # Zmień na swoją ścieżkę
    OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY_TEG')

# 3. Inicjalizacja LLM
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    openai_api_key=Config.OPENAI_API_KEY,
    temperature=0,
)

print("✅ LLM zainicjalizowany")

# 4. Połączenie z bazą danych
try:
    db = SQLDatabase.from_uri(f"sqlite:///{Config.DB_PATH}")
    print("✅ Połączono z bazą danych")
    
    # Sprawdź tabele
    print("\n📋 Dostępne tabele:")
    print(db.get_table_names())
    
    # Sprawdź strukturę tabeli logs
    print("\n🔍 Struktura tabeli 'logs':")
    print(db.get_table_info(["logs"]))
    
except Exception as e:
    print(f"❌ Błąd połączenia z bazą: {e}")

# 5. Stworzenie SQL agenta
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

sql_agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,  # Pokaż proces myślenia
    max_iterations=10,
    early_stopping_method="generate"
)

print("✅ SQL Agent utworzony")

# 6. Funkcja pomocnicza do testowania
def zapytaj_agenta(pytanie):
    """
    Zadaj pytanie SQL agentowi
    """
    print(f"\n🤔 PYTANIE: {pytanie}")
    print("="*60)
    
    try:
        odpowiedz = sql_agent.invoke({"input": pytanie})
        print(f"\n✅ ODPOWIEDŹ:")
        print(odpowiedz['output'])
        return odpowiedz
    except Exception as e:
        print(f"❌ BŁĄD: {e}")
        return None

# 7. Sprawdzenie danych w bazie
print("\n🔍 Sprawdzanie zawartości bazy...")

# Podstawowe statystyki
conn = sqlite3.connect(Config.DB_PATH)
cursor = conn.cursor()

# Liczba rekordów
cursor.execute("SELECT COUNT(*) FROM logs")
total_rows = cursor.fetchone()[0]
print(f"📊 Całkowita liczba rekordów: {total_rows}")

# Zakres dat
cursor.execute("SELECT MIN(date) as min_date, MAX(date) as max_date FROM logs")
date_range = cursor.fetchone()
print(f"📅 Zakres dat: {date_range[0]} - {date_range[1]}")

# Top 5 użytkowników
cursor.execute("SELECT srcname, COUNT(*) as count FROM logs GROUP BY srcname ORDER BY count DESC LIMIT 5")
top_users = cursor.fetchall()
print(f"\n👥 Top 5 użytkowników:")
for user, count in top_users:
    print(f"  - {user}: {count} sesji")

# Top 5 aplikacji
cursor.execute("SELECT app, COUNT(*) as count FROM logs GROUP BY app ORDER BY count DESC LIMIT 5")
top_apps = cursor.fetchall()
print(f"\n📱 Top 5 aplikacji:")
for app, count in top_apps:
    print(f"  - {app}: {count} sesji")

conn.close()

# 8. Przykładowe pytania do testowania
print("\n🧪 TESTY AGENTA")
print("="*60)

# Test 1: Podstawowe zapytanie
zapytaj_agenta("Ile rekordów jest w tabeli logs?")

# Test 2: Zapytanie o użytkownika
zapytaj_agenta("Pokaż mi 5 najaktywniejszych użytkowników")

# Test 3: Zapytanie o aplikacje
zapytaj_agenta("Które aplikacje są najczęściej używane?")

# Test 4: Zapytanie o czas
zapytaj_agenta("Jaki jest całkowity czas spędzony przez użytkownika Dawid w aplikacjach?")

# Test 5: Zapytanie o kategorie
zapytaj_agenta("Pokaż podział na kategorie aplikacji (appcat)")

# 9. Interaktywna sesja
def interaktywna_sesja():
    """
    Uruchom interaktywną sesję z agentem
    """
    print("\n🚀 INTERAKTYWNA SESJA Z SQL AGENTEM")
    print("Wpisz 'quit' aby zakończyć")
    print("="*60)
    
    while True:
        pytanie = input("\n❓ Twoje pytanie: ")
        
        if pytanie.lower() in ['quit', 'exit', 'koniec']:
            print("👋 Do widzenia!")
            break
            
        if pytanie.strip():
            zapytaj_agenta(pytanie)

# Uruchom interaktywną sesję (odkomentuj poniżej)
# interaktywna_sesja()

# 10. Przydatne funkcje pomocnicze
def pokaz_strukture_tabeli():
    """Pokaż strukturę tabeli logs"""
    conn = sqlite3.connect(Config.DB_PATH)
    cursor = conn.cursor()
    
    cursor.execute("PRAGMA table_info(logs)")
    columns = cursor.fetchall()
    
    print("\n📋 STRUKTURA TABELI 'logs':")
    print("-" * 60)
    print(f"{'Kolumna':<20} {'Typ':<15} {'Nullable':<10}")
    print("-" * 60)
    
    for col in columns:
        nullable = "NO" if col[3] else "YES"
        print(f"{col[1]:<20} {col[2]:<15} {nullable:<10}")
    
    conn.close()

def pokaz_przykladowe_dane(limit=5):
    """Pokaż przykładowe dane z tabeli"""
    conn = sqlite3.connect(Config.DB_PATH)
    
    df = pd.read_sql_query(f"SELECT * FROM logs LIMIT {limit}", conn)
    print(f"\n📊 PRZYKŁADOWE DANE ({limit} rekordów):")
    print("="*100)
    print(df.to_string())
    
    conn.close()

# Uruchom funkcje pomocnicze
pokaz_strukture_tabeli()
pokaz_przykladowe_dane(3)

print("\n🎉 SQL Agent jest gotowy do użycia!")
print("💡 Użyj funkcji zapytaj_agenta('twoje pytanie') aby zadać pytanie")
print("💡 Lub uruchom interaktywna_sesja() dla trybu interaktywnego")       

✅ LLM zainicjalizowany
✅ Połączono z bazą danych

📋 Dostępne tabele:
['logs']

🔍 Struktura tabeli 'logs':

CREATE TABLE logs (
	date TEXT, 
	time TEXT, 
	eventtime TEXT, 
	logid TEXT, 
	srcip TEXT, 
	srcname TEXT, 
	srcport INTEGER, 
	dstip TEXT, 
	dstport INTEGER, 
	proto INTEGER, 
	action TEXT, 
	policyname TEXT, 
	service TEXT, 
	transport TEXT, 
	appid TEXT, 
	app TEXT, 
	appcat TEXT, 
	apprisk TEXT, 
	duration INTEGER, 
	sentbyte INTEGER, 
	rcvdbyte INTEGER, 
	sentpkt INTEGER, 
	rcvdpkt INTEGER, 
	shapersentname TEXT, 
	osname TEXT, 
	mastersrcmac TEXT
)

/*
3 rows from logs table:
date	time	eventtime	logid	srcip	srcname	srcport	dstip	dstport	proto	action	policyname	service	transport	appid	app	appcat	apprisk	duration	sentbyte	rcvdbyte	sentpkt	rcvdpkt	shapersentname	osname	mastersrcmac
2025-05-16	10:19:02	1747383542538331649	0000000013	10.22.7.133	Dawid-s-S23	33232	57.144.112.141	443	6	close	LAN2WAN	HTTPS	33232	15832	Facebook	Social.Media	medium	1	2070	1707	9	7	medium-priority	Andr

/var/folders/g0/0_3w3d415fb5_p5bvx1g2l9r0000gp/T/ipykernel_19864/792026512.py:40: LangChainDeprecationWarning: The method `SQLDatabase.get_table_names` was deprecated in langchain-community 0.0.1 and will be removed in 1.0. Use :meth:`~get_usable_table_names` instead.
  print(db.get_table_names())


Action: sql_db_list_tables  
Action Input: ""  logsI need to check the schema of the "logs" table to understand its structure and then count the records in it.  
Action: sql_db_schema  
Action Input: "logs"  
CREATE TABLE logs (
	date TEXT, 
	time TEXT, 
	eventtime TEXT, 
	logid TEXT, 
	srcip TEXT, 
	srcname TEXT, 
	srcport INTEGER, 
	dstip TEXT, 
	dstport INTEGER, 
	proto INTEGER, 
	action TEXT, 
	policyname TEXT, 
	service TEXT, 
	transport TEXT, 
	appid TEXT, 
	app TEXT, 
	appcat TEXT, 
	apprisk TEXT, 
	duration INTEGER, 
	sentbyte INTEGER, 
	rcvdbyte INTEGER, 
	sentpkt INTEGER, 
	rcvdpkt INTEGER, 
	shapersentname TEXT, 
	osname TEXT, 
	mastersrcmac TEXT
)

/*
3 rows from logs table:
date	time	eventtime	logid	srcip	srcname	srcport	dstip	dstport	proto	action	policyname	service	transport	appid	app	appcat	apprisk	duration	sentbyte	rcvdbyte	sentpkt	rcvdpkt	shapersentname	osname	mastersrcmac
2025-05-16	10:19:02	1747383542538331649	0000000013	10.22.7.133	Dawid-s-S23	33232	57.144.112.141	4